In [ ]:
import xarray as xr
import panel as pn
import matplotlib.pyplot as plt
pn.extension()

# Open the zarr store
ds = xr.open_zarr("INCA.zarr")

# Select one chunk (example indices, adjust based on chunking)
chunk_data = ds["T2M"].isel(x=slice(0,100), y=slice(0,100), time=slice(0,720)).load()

def plot_time_slice(t):
    fig, ax = plt.subplots(figsize=(6,6))
    chunk_data.isel(time=t).plot(ax=ax)
    ax.set_title(f"Time index: {t}")
    plt.close(fig)  # Prevent double display
    return fig

time_slider = pn.widgets.IntSlider(start=0, end=chunk_data.time.size - 1, step=1, value=0)

mpl_pane = pn.pane.Matplotlib(plot_time_slice(time_slider.value), tight=True)

def update_plot(event):
    mpl_pane.object = plot_time_slice(event.new)

time_slider.param.watch(update_plot, 'value')

layout = pn.Column(time_slider, mpl_pane)
layout.servable()
layout

In [ ]:
chunk_data